#### 토스 커뮤니티는 동적으로 생성되므로, selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [64]:
import datetime
from dateutil.relativedelta import relativedelta

In [2]:
import time

In [3]:
import FinanceDataReader as fdr

In [4]:
from data_loader import load_sector_data

In [5]:
kospi = fdr.StockListing('KOSPI')
kospi.head(1)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,56200,1,100,0.18,56100,56500,55900,2194654,123260889300,335501779310000,5969782550,STK


In [6]:
kospi_list = kospi[['Code','Name']].head()

In [7]:
code = kospi_list[kospi_list['Name'] == '삼성전자']['Code'].values[0]

In [8]:
code

'005930'

In [9]:
# 종목명으로 업종코드 찾기

def search_code_by_name(market, name) :
  market_list = ['KOSPI','KOSDAQ','ETF']
  if market in market_list:
    if market in ['KOSPI','KOSDAQ']:
      df = fdr.StockListing(market)[['Code','Name']]
    else:
      df = fdr.StockListing('ETF/KR')[['Symbol','Name']]
  else:
    raise ValueError('시장입력오류')
  
  code = df[df['Name'] == name]['Code'].values[0]
  
  return code   
  

In [83]:
def code_list_by_market(market) :
  market_list = ['KOSPI','KOSDAQ','ETF']
  if market in market_list:
    if market in ['KOSPI','KOSDAQ']:
      df = fdr.StockListing(market).sort_values(by='Marcap', ascending=False).head(5)
    else:
      df = fdr.StockListing('ETF/KR')[['Symbol','Name']]
  else:
    raise ValueError('시장입력오류')
  
  code_list = df['Code'].tolist()
  return code_list

In [82]:
fdr.StockListing('KOSPI').sort_values(by='Marcap', ascending=False).head(5)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,55700,2,-400,-0.71,56100,56500,55200,10502816,584917256700,332516888035000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,207500,1,2500,1.22,208000,212500,206500,3742728,782465580500,151060490737500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,350000,2,-8500,-2.37,356000,357500,348500,163515,57326309500,81900000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,1011000,1,5000,0.50,1001000,1020000,999000,25028,25249684000,71956914000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,225000,1,12000,5.63,215000,227000,215000,1320309,294050720000,47118642975000,209416191,STK


In [84]:
code_list = code_list_by_market('KOSPI')
code_list

['005930', '000660', '373220', '207940', '005380']

In [10]:
samsung = search_code_by_name('KOSPI', '삼성전자')
samsung

'005930'

In [11]:
url = f'https://tossinvest.com/stocks/A005930/community'

In [51]:
# 웹드라이버 설정
driver = webdriver.Chrome()  # 또는 다른 웹드라이버 사용
driver.get(url)

In [13]:
# 스크롤을 내리며 데이터 로드
last_height = driver.execute_script("return document.body.scrollHeight")

In [65]:
# 오늘 날짜 구하기
today = datetime.datetime.today()
today_str = today.strftime('%Y-%m-%d')

In [ ]:
code_list = code_list_by_market('KOSPI')
code_list

In [78]:
# 종목명으로 종목 코드를 찾는 함수
def search_code_by_name(name):
  kospi_df = fdr.StockListing('KOSPI')
  kosdaq_df = fdr.StockListing('KOSDAQ')
  
  all_stocks = pd.concat([kospi_df,kosdaq_df])
  
  code = all_stocks[all_stocks['Name'] == name]['Code'].values[0]
  
  return code

In [79]:
tmp = search_code_by_name('삼성전자')
tmp

'005930'

In [91]:
def crawling_community(market=None, start_date=None, end_date=today_str):
    
    if start_date is None :
        raise ValueError('시작일은 반드시 입력해야 합니다.')
    
    code_list = code_list_by_market(market)
    all_post_data = []
    
    for code in code_list:
        
        url = f'https://tossinvest.com/stocks/A{code}/community'
        # 웹드라이버 초기화
        print(url)
        
        options = Options()
        options.add_argument('--headless')  # 헤드리스 모드
        driver = webdriver.Chrome(options=options)
        
        driver.get(url)  # 실제 URL로 변경

        WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#stock-content > div > div > section > section > ul > div > div'))
            )
        # 버튼 요소 찾기
        button = driver.find_element(By.CSS_SELECTOR, '#stock-content > div > div > section > section > section > button')

        # data-contents-label 속성 값 확인
        label = button.get_attribute('data-contents-label')

        # '인기순'이라면 버튼 클릭하여 '최신순'으로 변경
        if label != '최신순':
            button.click()
            time.sleep(2)  # 변경 후 로딩 시간 대기
            
        # 게시글 리스트를 저장할 리스트
        all_contents = []
        post_nickname = []
        post_date = []

        # 날짜 기준 설정
        start_date = pd.to_datetime(start_date).date()  # tz-naive로 설정
        end_date = pd.to_datetime(end_date).date()

        # 스크롤 반복을 위한 변수
        new_posts_found = True
        processed_indices = set()

        while new_posts_found:    
            # 게시글 리스트 추출
            all_posts = driver.find_elements(By.CSS_SELECTOR, '#stock-content > div > div > section > section > ul > div > div')  # 실제 클래스명에 맞게 수정

            # article 태그가 포함된 div만 필터링
            filtered_posts = [post for post in all_posts if post.find_elements(By.TAG_NAME, 'article')]

            new_posts_found = False  # 새로운 게시글 발견 여부 초기화
            
            for post in filtered_posts:
                # data-index 추출
                index = post.get_attribute('data-index')
                
                # print(index)
                # 이미 처리한 인덱스인지 확인
                if index in processed_indices:
                    continue  # 이미 처리한 게시글은 건너뜀
                
                content = post.find_element(By.CSS_SELECTOR, 'article > div > a > span:nth-of-type(2) > span ')
                datetime = post.find_element(By.CSS_SELECTOR, 'article > div > header > div > label > time')
                date = datetime.get_attribute('datetime')
                nickname = post.find_element(By.CSS_SELECTOR, 'article > div > header > div > label > span:nth-child(1)')
                
                # print(content.text)
                # print(nickname.text)
                # 날짜 변환 (tz-naive)
                date_obj = pd.to_datetime(date).tz_localize(None).date()  # 날짜만 추출
                
                # 날짜 필터링
                if start_date <= date_obj <= end_date:
                    all_contents.append(content.text)
                    post_nickname.append(nickname.text)
                    post_date.append(date)
                    processed_indices.add(index)  # 인덱스를 집합에 추가
                    new_posts_found = True  # 새로운 게시글이 발견됨

            # 스크롤을 내림
            driver.execute_script("window.scrollBy(0, 3000);")
            time.sleep(2)  # 로딩 시간 대기

            # 특정 요소가 로드될 때까지 대기 (예: 게시글 목록의 특정 요소)
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#stock-content > div > div > section > section > ul > div > div'))
                )
            except Exception as e:
                print("로딩 대기 중 오류 발생:", e)
                break  # 로딩 중 오류가 발생하면 루프 종료

        # 데이터프레임 생성
        post_infos = {
            '닉네임': post_nickname,
            '날짜': post_date,
            '내용': all_contents,
            '종목': code
        }

        post_df = pd.DataFrame(post_infos)
        all_post_data.append(post_df)
        driver.quit()
        
    final_df = pd.concat(all_post_data, ignore_index=True)    
    return final_df

(시가총액 상위 5종목) - 8분30초( 약 1700건)

In [92]:
result = crawling_community('KOSPI', start_date='2024-12-30')

https://tossinvest.com/stocks/A005930/community
https://tossinvest.com/stocks/A000660/community


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6749B80D5+2992373]
	(No symbol) [0x00007FF67464BFD0]
	(No symbol) [0x00007FF6744E590A]
	(No symbol) [0x00007FF6744EC46F]
	(No symbol) [0x00007FF6744EF49F]
	(No symbol) [0x00007FF674580840]
	(No symbol) [0x00007FF67455F36A]
	(No symbol) [0x00007FF67457F584]
	(No symbol) [0x00007FF67455F113]
	(No symbol) [0x00007FF67452A918]
	(No symbol) [0x00007FF67452BA81]
	GetHandleVerifier [0x00007FF674A16A2D+3379789]
	GetHandleVerifier [0x00007FF674A2C32D+3468109]
	GetHandleVerifier [0x00007FF674A20043+3418211]
	GetHandleVerifier [0x00007FF6747AC78B+847787]
	(No symbol) [0x00007FF67465757F]
	(No symbol) [0x00007FF674652FC4]
	(No symbol) [0x00007FF67465315D]
	(

In [87]:
result

,닉네임,날짜,내용,종목
0,단타를노린다,2025-01-10T12:42:05+09:00,제바알…. 좀 올라라 ㅠㅠㅠㅠㅠ,005930
1,아르네슬롯,2025-01-10T12:41:48+09:00,한 달 기다려서 4만원 힘들었네요,005930
2,두두다14,2025-01-10T12:41:35+09:00,가니깐,005930
3,오늘은내가고점판독기,2025-01-10T12:41:12+09:00,미장만 하던 주린이는 삼전 들어오니 그저 놀랍습니다\n여기서 다들 무슨 인내를 쌓고...,005930
4,주식작전사령부,2025-01-10T12:40:12+09:00,꾸준매 미라클 가즈아!,005930
...,...,...,...,...
1611,다시는숏치비않겠습니다,2025-01-09T09:12:52+09:00,머선일이고,005380
1612,빨간불좋아요좋아,2025-01-09T09:12:12+09:00,드디어 오르네,005380
1613,진호돌,2025-01-09T09:07:44+09:00,머선129,005380
1614,어어어어밀지마라,2025-01-09T09:06:08+09:00,네이따ㅏㅏ,005380


In [88]:
result.to_csv(f'practice_1.csv', index=False, encoding='utf-8-sig')

In [59]:
post_df

,닉네임,날짜,내용
0,구부랑길고양이,2025-01-10T11:32:47+09:00,진짜 앞자리 6을 못보네
1,부자71,2025-01-10T11:31:00+09:00,세상 상황을 아는것도 주식과 연결되듯이\n중국은 내정간섭중입니다.\n정치인과 결탁되...
2,슈랭슈랭,2025-01-10T11:30:20+09:00,울 삼전이는 시간이 약이네요.스트레스 지지~\n어제 뉴스에 댓글에 댓글보고 혼자 빵...
3,공동우승,2025-01-10T11:30:16+09:00,4만 가즈아~~~
4,쓉거지키우기,2025-01-10T11:30:16+09:00,얘네 뭐하냐
...,...,...,...
2956,Youngest,2025-01-06T07:47:52+09:00,성과급 자랑하지말자 소액 주주들 피똥싸는 중이다
2957,단타는몸에좋다,2025-01-06T07:14:48+09:00,단체로 정치에 미쳐서 삼성 이야기는 안하고 정치얘기밖에 없네
2958,Youngest,2025-01-06T05:49:34+09:00,나 또 왔다 ㅋㅋ
2959,자민당,2025-01-06T02:00:26+09:00,쩝.


In [32]:
print(len(post_df))

747


In [33]:
# post_df = post_df.drop_duplicates()

In [61]:
post_df.to_csv(f'practice_Toss_{code}.csv', index=False, encoding='utf-8-sig')